In [1]:
%pip install openAI
%pip install langchain
%pip install typing-inspect==0.8.0 typing_extensions==4.5.0
%pip install pydantic==1.10.8

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
from langchain import *
from pydantic import *
import os 
import pandas as pd 
with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1] #for some reason there's a new line, so we remove it



# Let's try LangChain

Literally just grabbed the example from the website and modified. still have no idea how this works but whatever (for now) 


## Putting everything together

In [4]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List



# Define your desired data structure.
class extracted_data(BaseModel):
    descriptions: list = Field(description="The descriptions for the shape. This should be a list of one or more strings.")
    
    





input =pd.read_csv("data/pre/5_1.csv")

parser = PydanticOutputParser(pydantic_object=extracted_data)

info_query_generic = "Here's a partial transcript of someone describing an image to someone else. Extract verbatim the parts of the text that describe of the image. \
Do not include fillers like 'it looks like' or 'sorta'  \
  or conversational words that aren't descriptive. \n\n \
  As an example, if the transcript was 'It looks like a magician, and, uhh, I think he's got a rabbit.', the response would be ['a magician','he's got a rabbit'].\n\n \
     As an example, if the transcript was 'yes, sorry, big triangle is on the right, like an arm', the response would be ['big triangle is on the right, like an arm'].\n\n \
      Return just the description in list format. Here's the text:\n\n "
#model_name = "gpt-3.5-turbo"
model_name="gpt-4"
temperature = 0
model = ChatOpenAI(temperature=temperature)


def code_data_with_langchain(df, model, parser, info_query_generic): 
  langchain_prompt = PromptTemplate(
    template="{query}\n",
    input_variables=["query"]
  )
  system_prompt = "You are a competent researcher who answers user queries."

  res = []
  #for index in range(2):
  for index, _ in df.iterrows():
    print(index)
    info_query = info_query_generic + df["text"][index] + "\n\n"
    _input = langchain_prompt.format_prompt(query=info_query)
    #print(_input)
    full_prompt=[SystemMessage(content=system_prompt), HumanMessage(content=_input.to_string())]
    #print(full_prompt)
    output = model(full_prompt)
    #parsed_output = parser.parse(output)
    string_output=output.content
    print(string_output)
    res.append(string_output)
  return res


res = code_data_with_langchain(input, model, parser,  info_query_generic)

output=input
output['gpt_out']=res
output.to_csv("data/post/5_1.csv")





0
['small triangle bottom right, falling']
1
['the one with square and triangle', 'falling off cliff', 'triangle on right', 'small foot left side', 'the square is standing', 'triangle is leaning', 'small leg on bottom left']
2
['leaning to the left', 'square in the middle of two hands', 'falling yes', 'two arms up']
3
['the sitting one', '45 degree legs', 'legs on left side', 'head is square', 'left side is bent up', 'yes on butt', 'no hands', 'long edge 45 degree', 'on left side', 'legs on left body on right']
4
['square on top', 'small edge on left side', 'bigger edge on right side', 'straight up', 'pointing right', 'two big edges', 'pac man mouth', 'yes jiggly tail', 'yes on left']
5
['big block figure', 'stubby fat guy', 'belly', 'arm pointing right', 'standing']
6
['looks like it's bending', 'right arm', 'back straight', 'square head', 'arm to right', 'yes bendy', 'zagging legs', 'one edge on right side', 'two edges sorry']
7
['big body', 'small edge on bottom right side', 'no arm

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Mon, 10 Jul 2023 17:14:29 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e4a69e29eca168e-SJC', 'alt-svc': 'h3=":443"; ma=86400'}.


['someone stretching', 'legs and feet are flat against the ground, facing to the left', 'arms are behind the torso to the right of the frame', 'guy is facing left overall', 'looks kind of like a lowercase "j"']
158
['a ballerina or a martial artist', 'kneeling on the ground, arms outstretched to either side', 'Facing right, feet pointed left', 'You can see one foot and both hands']
159
['reading or praying', 'facing to the right', 'kneeling, with his knees and shins flat against the bottom', 'tips of the feet are pointing up']
160
['Both arms are over the head, one in each direction', 'The guy is facing to the right', 'the rest looks like mountains', 'Or a graph']
161
['This one doesn't show legs or feet', 'The guy is facing left', 'He may have something in his hands, which are slightly outstretched from his body', 'It looks sort of like: -i']
162
['looks like a lowercase Y', 'person is sort of leaning to the left', 'arms lifted above the shoulders']
163
['facing right', 'Kneeling, but